In [7]:
# Cell 1: Setup and S&P 500 CIK Mapping from Wikipedia
import pandas as pd
import numpy as np
import json
import pickle
import os
import requests
from io import StringIO
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Data directories
SEC_DIR = os.path.join('..', 'data', 'raw', 'kaggle', 'sec_edgar')
FACTS_DIR = os.path.join(SEC_DIR, 'companyfacts')
DATA_DIR = os.path.join('..', 'data')

print(f'SEC data dir: {os.path.abspath(FACTS_DIR)}')
print(f'SEC files found: {len(os.listdir(FACTS_DIR))}')

# Load existing price data (created by notebook 01c)
with open(os.path.join(DATA_DIR, 'price_data.pkl'), 'rb') as f:
    price_data = pickle.load(f)
price_tickers = set(price_data.keys())
print(f'Price data: {len(price_tickers)} companies')

# Scrape S&P 500 table from Wikipedia to get CIK numbers
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {'User-Agent': 'Mozilla/5.0 (stock-prediction-ml project)'}
response = requests.get(url, headers=headers)
response.raise_for_status()
tables = pd.read_html(StringIO(response.text))
sp500_df = tables[0]

print(f'\nWikipedia S&P 500: {len(sp500_df)} companies')
print(f'Columns: {list(sp500_df.columns)}')

# Build ticker-to-CIK mapping
ticker_to_cik = {}
for _, row in sp500_df.iterrows():
    ticker_wiki = str(row['Symbol']).strip()
    ticker_yf = ticker_wiki.replace('.', '-')  # BRK.B -> BRK-B
    try:
        cik = str(int(row['CIK'])).zfill(10)
    except (ValueError, TypeError):
        continue
    json_path = os.path.join(FACTS_DIR, f'CIK{cik}.json')
    if os.path.exists(json_path):
        # Use yfinance-style ticker if it has price data
        if ticker_yf in price_tickers:
            ticker_to_cik[ticker_yf] = cik
        elif ticker_wiki in price_tickers:
            ticker_to_cik[ticker_wiki] = cik

print(f'\nS&P 500 tickers with SEC EDGAR + price data: {len(ticker_to_cik)}')

SEC data dir: c:\Users\chris\stock-prediction-ml\data\raw\kaggle\sec_edgar\companyfacts
SEC files found: 18848
Price data: 502 companies

Wikipedia S&P 500: 503 companies
Columns: ['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded']

S&P 500 tickers with SEC EDGAR + price data: 450


In [8]:
# Cell 2: SEC EDGAR Parser (unchanged from previous version)

# XBRL tag mapping: our metric name -> list of possible SEC tags (tried in order)
INCOME_TAGS = {
    'Total Revenue': [
        'RevenueFromContractWithCustomerExcludingAssessedTax',
        'Revenues',
        'SalesRevenueNet',
        'SalesRevenueGoodsNet',
        'RevenueFromContractWithCustomerIncludingAssessedTax',
    ],
    'Cost of Revenue': [
        'CostOfGoodsAndServicesSold',
        'CostOfRevenue',
        'CostOfGoodsSold',
    ],
    'Gross Profit': ['GrossProfit'],
    'Operating Income': ['OperatingIncomeLoss'],
    'Net Income': ['NetIncomeLoss'],
    'EPS Basic': ['EarningsPerShareBasic'],
    'EPS Diluted': ['EarningsPerShareDiluted'],
    'R&D Expense': ['ResearchAndDevelopmentExpense'],
    'SGA Expense': [
        'SellingGeneralAndAdministrativeExpense',
        'SellingAndMarketingExpense',
    ],
    'Interest Expense': [
        'InterestExpense',
        'InterestExpenseDebt',
    ],
    'Income Tax': ['IncomeTaxExpenseBenefit'],
}

BALANCE_TAGS = {
    'Total Assets': ['Assets'],
    'Current Assets': ['AssetsCurrent'],
    'Total Liabilities': ['Liabilities', 'LiabilitiesAndStockholdersEquity'],
    'Current Liabilities': ['LiabilitiesCurrent'],
    'Long Term Debt': ['LongTermDebtNoncurrent', 'LongTermDebt'],
    'Short Term Debt': ['ShortTermBorrowings', 'CommercialPaper'],
    'Stockholders Equity': ['StockholdersEquity'],
    'Cash': ['CashAndCashEquivalentsAtCarryingValue', 'CashCashEquivalentsAndShortTermInvestments'],
    'Shares Outstanding': ['CommonStockSharesOutstanding'],
}

CASHFLOW_TAGS = {
    'Operating Cash Flow': [
        'NetCashProvidedByUsedInOperatingActivities',
        'NetCashProvidedByUsedInOperatingActivitiesContinuingOperations',
    ],
    'Capital Expenditures': [
        'PaymentsToAcquirePropertyPlantAndEquipment',
        'PaymentsToAcquireProductiveAssets',
    ],
}

def extract_quarterly_values(facts, tag_names, is_instant=False):
    """Extract single-quarter values from SEC XBRL data."""
    gaap = facts.get('us-gaap', {})

    for tag in tag_names:
        if tag not in gaap:
            continue

        units = gaap[tag].get('units', {})
        if 'USD' in units:
            entries = units['USD']
        elif 'shares' in units:
            entries = units['shares']
        elif 'USD/shares' in units:
            entries = units['USD/shares']
        else:
            continue

        quarterly = [e for e in entries if e.get('form') == '10-Q']
        if not quarterly:
            continue

        values = {}
        for entry in quarterly:
            end_date = pd.Timestamp(entry['end'])

            if is_instant:
                values[end_date] = entry['val']
            else:
                start = entry.get('start')
                if start:
                    days = (datetime.strptime(entry['end'], '%Y-%m-%d') -
                            datetime.strptime(start, '%Y-%m-%d')).days
                    if 85 <= days <= 100:
                        values[end_date] = entry['val']

        if values:
            return values, tag

    return {}, None


def load_company_financials(ticker, cik, facts_dir):
    """Load and parse SEC financial data for one company."""
    json_path = os.path.join(facts_dir, f'CIK{cik}.json')
    with open(json_path) as f:
        data = json.load(f)

    facts = data['facts']

    income_data = {}
    for metric_name, tag_list in INCOME_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=False)
        if values:
            income_data[metric_name] = values

    balance_data = {}
    for metric_name, tag_list in BALANCE_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=True)
        if values:
            balance_data[metric_name] = values

    cashflow_data = {}
    for metric_name, tag_list in CASHFLOW_TAGS.items():
        values, tag_used = extract_quarterly_values(facts, tag_list, is_instant=False)
        if values:
            cashflow_data[metric_name] = values

    if income_data:
        income_df = pd.DataFrame(income_data)
        income_df.index = pd.to_datetime(income_df.index)
        income_df = income_df.sort_index()
        income_df = income_df.T
    else:
        income_df = pd.DataFrame()

    all_balance = {**balance_data, **cashflow_data}
    if all_balance:
        balance_df = pd.DataFrame(all_balance)
        balance_df.index = pd.to_datetime(balance_df.index)
        balance_df = balance_df.sort_index()
        balance_df = balance_df.T
    else:
        balance_df = pd.DataFrame()

    return {
        'ticker': ticker,
        'quarterly_income': income_df,
        'quarterly_balance': balance_df,
        'info': {'symbol': ticker},
    }

print('SEC EDGAR parser defined')
print(f'Income tags: {len(INCOME_TAGS)} metrics')
print(f'Balance tags: {len(BALANCE_TAGS)} metrics')
print(f'Cash flow tags: {len(CASHFLOW_TAGS)} metrics')
print(f'Total: {len(INCOME_TAGS) + len(BALANCE_TAGS) + len(CASHFLOW_TAGS)} raw metrics')

SEC EDGAR parser defined
Income tags: 11 metrics
Balance tags: 9 metrics
Cash flow tags: 2 metrics
Total: 22 raw metrics


In [9]:
# Cell 3: Extract Financial Data for ALL S&P 500 Companies
print(f'Parsing SEC financial data for {len(ticker_to_cik)} S&P 500 companies...')
print('(quality thresholds: income_metrics >= 4, quarters >= 8)\n')

financial_data = []
skipped = []
errors = []

for i, (ticker, cik) in enumerate(sorted(ticker_to_cik.items()), 1):
    try:
        company = load_company_financials(ticker, cik, FACTS_DIR)

        income_metrics = len(company['quarterly_income'])
        balance_metrics = len(company['quarterly_balance'])
        quarters = company['quarterly_income'].shape[1] if not company['quarterly_income'].empty else 0

        # Require meaningful filing history for quality features
        if income_metrics >= 4 and quarters >= 8:
            financial_data.append(company)
            if i % 50 == 0 or i == len(ticker_to_cik):
                print(f'  [{i}/{len(ticker_to_cik)}] {ticker}: '
                      f'{income_metrics} income + {balance_metrics} balance, {quarters} quarters')
        else:
            skipped.append(f'{ticker} ({income_metrics} income, {quarters} quarters)')

    except Exception as e:
        errors.append(f'{ticker}: {e}')

print(f'\n--- Results ---')
print(f'Loaded: {len(financial_data)} companies')
print(f'Skipped: {len(skipped)} (insufficient data)')
print(f'Errors: {len(errors)}')

if skipped:
    print(f'\nSkipped companies: {skipped[:20]}')
    if len(skipped) > 20:
        print(f'  ... and {len(skipped) - 20} more'
              )
if errors:
    print(f'\nErrors: {errors[:10]}')

Parsing SEC financial data for 450 S&P 500 companies...
(quality thresholds: income_metrics >= 4, quarters >= 8)

  [50/450] BRK-B: 6 income + 6 balance, 52 quarters
  [100/450] CVS: 9 income + 11 balance, 53 quarters
  [150/450] FCX: 9 income + 9 balance, 54 quarters
  [200/450] IBM: 10 income + 10 balance, 53 quarters
  [250/450] LYB: 10 income + 11 balance, 45 quarters
  [300/450] NVR: 6 income + 6 balance, 50 quarters
  [350/450] RMD: 11 income + 11 balance, 50 quarters
  [400/450] TSN: 9 income + 8 balance, 50 quarters
  [450/450] ZTS: 8 income + 11 balance, 42 quarters

--- Results ---
Loaded: 443 companies
Skipped: 7 (insufficient data)
Errors: 0

Skipped companies: ['AZO (9 income, 1 quarters)', 'BLK (6 income, 7 quarters)', 'COST (9 income, 2 quarters)', 'DPZ (9 income, 1 quarters)', 'KR (0 income, 0 quarters)', 'PEP (0 income, 0 quarters)', 'PSKY (7 income, 3 quarters)']


In [10]:
# Cell 4: Data Quality Report

print('=' * 60)
print('DATA QUALITY REPORT')
print('=' * 60)

print(f'\n--- Financial Data ---')
print(f'Companies: {len(financial_data)}')

# Sector distribution (cross-reference with sector_data)
try:
    with open(os.path.join(DATA_DIR, 'sector_data.pkl'), 'rb') as f:
        sector_data = pickle.load(f)
    sector_counts = {}
    for company in financial_data:
        sector_info = sector_data.get(company['ticker'], {})
        sector = sector_info.get('sector', 'Unknown')
        sector_counts[sector] = sector_counts.get(sector, 0) + 1
    print(f'\nCompanies per sector:')
    for sector, count in sorted(sector_counts.items(), key=lambda x: -x[1]):
        print(f'  {sector:<30s} {count:>3}')
except FileNotFoundError:
    print('(sector_data.pkl not found)')

# Metric coverage
all_income_metrics = set()
all_balance_metrics = set()
for company in financial_data:
    all_income_metrics.update(company['quarterly_income'].index.tolist())
    all_balance_metrics.update(company['quarterly_balance'].index.tolist())

print(f'\n--- Metric Coverage (% of companies) ---')
for metric in sorted(all_income_metrics):
    count = sum(1 for c in financial_data if metric in c['quarterly_income'].index)
    pct = count / len(financial_data) * 100
    print(f'  {metric:<30s} {count:>3}/{len(financial_data)} ({pct:.0f}%)')
for metric in sorted(all_balance_metrics):
    count = sum(1 for c in financial_data if metric in c['quarterly_balance'].index)
    pct = count / len(financial_data) * 100
    print(f'  {metric:<30s} {count:>3}/{len(financial_data)} ({pct:.0f}%)')

# Quarters per company distribution
quarters_list = []
for company in financial_data:
    q = company['quarterly_income'].shape[1] if not company['quarterly_income'].empty else 0
    quarters_list.append(q)
print(f'\n--- Quarters per Company ---')
print(f'  Min: {min(quarters_list)}, Max: {max(quarters_list)}, '
      f'Mean: {np.mean(quarters_list):.1f}, Median: {np.median(quarters_list):.0f}')

DATA QUALITY REPORT

--- Financial Data ---
Companies: 443

Companies per sector:
  Industrials                     68
  Information Technology          65
  Financials                      65
  Health Care                     55
  Consumer Discretionary          45
  Utilities                       30
  Real Estate                     30
  Materials                       22
  Consumer Staples                22
  Communication Services          21
  Energy                          20

--- Metric Coverage (% of companies) ---
  Cost of Revenue                311/443 (70%)
  EPS Basic                      433/443 (98%)
  EPS Diluted                    433/443 (98%)
  Gross Profit                   185/443 (42%)
  Income Tax                     437/443 (99%)
  Interest Expense               378/443 (85%)
  Net Income                     436/443 (98%)
  Operating Income               374/443 (84%)
  R&D Expense                    137/443 (31%)
  SGA Expense                    302/443 (68%)

In [11]:
# Cell 5: Save Expanded Data

# Save financial data
with open(os.path.join(DATA_DIR, 'financial_data.pkl'), 'wb') as f:
    pickle.dump(financial_data, f)

# Save ticker list
final_tickers = [c['ticker'] for c in financial_data]
with open(os.path.join(DATA_DIR, 'tickers.pkl'), 'wb') as f:
    pickle.dump(final_tickers, f)

print('Data saved:')
print(f'  financial_data.pkl - {len(financial_data)} companies')
print(f'  tickers.pkl - {len(final_tickers)} tickers')
print(f'\nNote: price_data.pkl is unchanged (502 companies from notebook 01c)')

Data saved:
  financial_data.pkl - 443 companies
  tickers.pkl - 443 tickers

Note: price_data.pkl is unchanged (502 companies from notebook 01c)


In [12]:
# Cell 6: Verification

with open(os.path.join(DATA_DIR, 'financial_data.pkl'), 'rb') as f:
    test_financials = pickle.load(f)

# Check first company
sample = test_financials[0]
ticker = sample['ticker']

print(f'=== Verification: {ticker} ===')
print(f'\nQuarterly Income ({sample["quarterly_income"].shape}):')
print(f'  Metrics: {list(sample["quarterly_income"].index)}')
print(f'\nQuarterly Balance ({sample["quarterly_balance"].shape}):')
print(f'  Metrics: {list(sample["quarterly_balance"].index)}')

# Verify all companies
print(f'\n=== All {len(test_financials)} companies ===')
for c in test_financials[:10]:
    t = c['ticker']
    iq = c['quarterly_income'].shape[1] if not c['quarterly_income'].empty else 0
    im = len(c['quarterly_income'])
    bm = len(c['quarterly_balance'])
    print(f'  {t:<6s} {im} income + {bm} balance metrics, {iq} quarters')
if len(test_financials) > 10:
    print(f'  ... and {len(test_financials) - 10} more companies')

print(f'\nData is ready for notebook 02 (feature engineering)!')

=== Verification: A ===

Quarterly Income ((10, 50)):
  Metrics: ['Total Revenue', 'Cost of Revenue', 'Operating Income', 'Net Income', 'EPS Basic', 'EPS Diluted', 'R&D Expense', 'SGA Expense', 'Interest Expense', 'Income Tax']

Quarterly Balance ((11, 68)):
  Metrics: ['Total Assets', 'Current Assets', 'Total Liabilities', 'Current Liabilities', 'Long Term Debt', 'Short Term Debt', 'Stockholders Equity', 'Cash', 'Shares Outstanding', 'Operating Cash Flow', 'Capital Expenditures']

=== All 443 companies ===
  A      10 income + 11 balance metrics, 50 quarters
  AAPL   11 income + 11 balance metrics, 53 quarters
  ABBV   9 income + 9 balance metrics, 42 quarters
  ABNB   7 income + 9 balance metrics, 18 quarters
  ACGL   6 income + 6 balance metrics, 50 quarters
  ACN    9 income + 9 balance metrics, 52 quarters
  ADBE   10 income + 10 balance metrics, 53 quarters
  ADSK   11 income + 9 balance metrics, 56 quarters
  AEE    7 income + 11 balance metrics, 53 quarters
  AES    8 income + 